# EcoHome Energy Advisor - Google Colab Quick Start

This notebook sets up and tests the EcoHome agent in Google Colab.

⚠️ **Ignore dependency conflict warnings** - they're from Colab's internal packages and won't affect functionality.

# EcoHome Energy Advisor - Google Colab Quick Start

This notebook sets up and tests the EcoHome agent in Google Colab.

⚠️ **Ignore dependency conflict warnings** - they're from Colab's internal packages and won't affect functionality.

## Step 1: Clone Repository and Install Dependencies

In [ ]:
# Clone repository
!git clone https://github.com/DipakBagal/udacity-ecohome-solutions.git
%cd udacity-ecohome-solutions

# Install dependencies (ignore version conflict warnings)
!pip install -q langchain>=0.1.0 langchain-openai>=0.0.5 langchain-community>=0.0.20 langgraph>=0.0.20 chromadb>=0.4.22 sqlalchemy>=2.0.0 python-dotenv>=1.0.0

print("✓ Installation complete!")

## Step 2: Set OpenAI API Key

**IMPORTANT:** Replace with your actual OpenAI API key!

In [ ]:
import os

# ⚠️ REPLACE WITH YOUR ACTUAL API KEY
os.environ['OPENAI_API_KEY'] = 'sk-proj-YOUR-KEY-HERE'

# Verify key is set
if os.environ.get('OPENAI_API_KEY', '').startswith('sk-'):
    print("✓ API key configured")
else:
    print("⚠️ WARNING: Please set your actual OpenAI API key above!")

## Step 3: Test Python Syntax (No API Needed)

In [ ]:
import sys
import ast

print("Testing Python files for syntax errors...\n")
print("=" * 60)

test_files = ['agent.py', 'tools.py', 'models/energy.py']
all_passed = True

for file_path in test_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()
        ast.parse(code)
        print(f"✓ {file_path}: Valid Python syntax")
    except SyntaxError as e:
        print(f"✗ {file_path}: Syntax error at line {e.lineno}")
        all_passed = False
    except Exception as e:
        print(f"? {file_path}: {str(e)}")
        all_passed = False

print("=" * 60)
if all_passed:
    print("\n✓ All syntax tests passed!")
else:
    print("\n⚠️ Some tests failed")

## Step 4: Initialize Database with Sample Data

Creates SQLite database with 90 days of energy usage and solar generation data.

In [ ]:
from models.energy import init_db, get_session, EnergyUsage, SolarGeneration
from datetime import datetime, timedelta
import random

print("Initializing database...")
init_db()

session = get_session()

# Generate 90 days of data
devices = ['hvac', 'water_heater', 'ev_charger', 'lighting', 'appliances']
start_date = datetime.now() - timedelta(days=90)

print("Generating sample data...")
for day in range(90):
    current_date = start_date + timedelta(days=day)
    
    # Energy usage (24 hourly records per device)
    for hour in range(24):
        timestamp = current_date + timedelta(hours=hour)
        for device in devices:
            # Realistic consumption patterns
            if device == 'hvac':
                consumption = random.uniform(2.5, 4.5)
            elif device == 'water_heater':
                consumption = random.uniform(1.0, 2.5)
            elif device == 'ev_charger':
                consumption = random.uniform(0, 7.0) if 22 <= hour or hour < 6 else 0
            elif device == 'lighting':
                consumption = random.uniform(0.3, 1.2) if 18 <= hour or hour < 7 else random.uniform(0.1, 0.4)
            else:  # appliances
                consumption = random.uniform(0.5, 2.0)
            
            usage = EnergyUsage(timestamp=timestamp, device=device, consumption_kwh=consumption)
            session.add(usage)
    
    # Solar generation (12 hours: 6 AM to 6 PM)
    for hour in range(6, 18):
        timestamp = current_date + timedelta(hours=hour)
        # Peak generation around noon
        hour_factor = 1 - abs(12 - hour) / 6
        generation = random.uniform(3.0, 6.0) * hour_factor
        efficiency = random.uniform(0.85, 0.95)
        
        solar = SolarGeneration(timestamp=timestamp, generation_kwh=generation, panel_efficiency=efficiency)
        session.add(solar)

session.commit()
session.close()

print(f"\n✓ Database initialized with 90 days of data")
print(f"  - Energy usage records: {90 * 24 * len(devices):,}")
print(f"  - Solar generation records: {90 * 12:,}")

## Step 5: Create Vector Store from Knowledge Documents

Processes 7 knowledge documents into ChromaDB for RAG retrieval.

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

print("Loading knowledge documents...")

documents_dir = "data/documents"
document_files = [
    "tip_device_best_practices.txt",
    "tip_energy_savings.txt",
    "hvac_optimization.txt",
    "smart_home_automation.txt",
    "renewable_energy_integration.txt",
    "seasonal_energy_management.txt",
    "energy_storage_optimization.txt"
]

all_texts = []
for filename in document_files:
    filepath = os.path.join(documents_dir, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
        all_texts.append(content)
        print(f"  ✓ Loaded {filename} ({len(content)} chars)")

# Split into chunks
print("\nSplitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = []
for text in all_texts:
    chunks.extend(text_splitter.split_text(text))

print(f"  ✓ Created {len(chunks)} chunks")

# Create embeddings and vector store
print("\nCreating vector store (this may take 1-2 minutes)...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("\n✓ Vector store created successfully!")
print(f"  - Total chunks indexed: {len(chunks)}")
print(f"  - Storage location: ./chroma_db")

## Step 6: Quick Agent Test

Tests the agent with a simple query to verify everything works.

In [ ]:
from agent import create_agent
from datetime import datetime

print("Creating agent...")
agent = create_agent()

print("\n" + "=" * 60)
print("Testing agent with sample query...")
print("=" * 60 + "\n")

query = "What are my top 3 energy-consuming devices today?"
print(f"User: {query}\n")

response = agent.chat(query)
print(f"Agent: {response}")

print("\n" + "=" * 60)
print("✓ Agent test complete!")
print("=" * 60)

## Step 7: Run Comprehensive Evaluation

Tests 8 different scenarios to validate all agent capabilities.

In [ ]:
# Test cases covering all capabilities
test_cases = [
    {
        "name": "Weather + Solar Analysis",
        "query": "What's the weather forecast for San Francisco this week, and how will it affect my solar generation?",
        "expected_tools": ["get_weather_forecast", "query_solar_generation"]
    },
    {
        "name": "EV Charging Optimization",
        "query": "When should I charge my electric vehicle to save money?",
        "expected_tools": ["get_electricity_prices", "search_energy_tips"]
    },
    {
        "name": "HVAC Efficiency",
        "query": "How can I make my HVAC system more efficient?",
        "expected_tools": ["search_energy_tips"]
    },
    {
        "name": "Usage Analysis",
        "query": "Show me my energy consumption for the past week",
        "expected_tools": ["query_energy_usage"]
    },
    {
        "name": "Seasonal Tips",
        "query": "What energy-saving strategies should I use this winter?",
        "expected_tools": ["search_energy_tips"]
    },
    {
        "name": "Battery Storage",
        "query": "Should I install battery storage with my solar panels?",
        "expected_tools": ["search_energy_tips"]
    },
    {
        "name": "Multi-Tool Query",
        "query": "Based on my usage last month and current electricity prices, how much can I save by shifting consumption to off-peak hours?",
        "expected_tools": ["query_energy_usage", "get_electricity_prices"]
    },
    {
        "name": "Price Analysis",
        "query": "What are the electricity prices for this week?",
        "expected_tools": ["get_electricity_prices"]
    }
]

print("\n" + "=" * 80)
print("COMPREHENSIVE EVALUATION - 8 Test Cases")
print("=" * 80 + "\n")

results = []
for i, test in enumerate(test_cases, 1):
    print(f"\n{'─' * 80}")
    print(f"Test {i}/8: {test['name']}")
    print(f"{'─' * 80}")
    print(f"\nQuery: {test['query']}\n")
    
    try:
        response = agent.chat(test['query'])
        print(f"Response: {response}\n")
        
        # Check if response is reasonable
        success = len(response) > 50 and "error" not in response.lower()
        results.append({"test": test['name'], "success": success})
        
        if success:
            print("✓ Test passed")
        else:
            print("⚠️ Response seems too short or contains errors")
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        results.append({"test": test['name'], "success": False})

# Summary
print("\n" + "=" * 80)
print("EVALUATION SUMMARY")
print("=" * 80)

passed = sum(1 for r in results if r['success'])
total = len(results)

for result in results:
    status = "✓" if result['success'] else "✗"
    print(f"{status} {result['test']}")

print(f"\n{'─' * 80}")
print(f"Total: {passed}/{total} tests passed ({passed/total*100:.1f}%)")
print("=" * 80)

## ✓ Testing Complete!

If all tests passed, your EcoHome Energy Advisor is working correctly!

### Next Steps:

1. **Try custom queries**: Create new cells and test your own questions
2. **Explore notebooks**: Check out the other notebooks in the repo
3. **Local setup**: If needed, follow README.md for local installation

### Common Issues:

- **API Rate Limits**: If you hit OpenAI rate limits, wait a minute and retry
- **Database locked**: Restart runtime if SQLite shows lock errors
- **Module not found**: Rerun Step 1 to reinstall dependencies

# EcoHome Energy Advisor - Google Colab Quick Start

This notebook sets up and tests the EcoHome agent in Google Colab.

⚠️ **Ignore dependency conflict warnings** - they're from Colab's internal packages and won't affect functionality.

## Step 1: Clone Repository and Install Dependencies

In [ ]:
# Clone repository
!git clone https://github.com/DipakBagal/udacity-ecohome-solutions.git
%cd udacity-ecohome-solutions

# Install dependencies (ignore version conflict warnings)
!pip install -q langchain>=0.1.0 langchain-openai>=0.0.5 langchain-community>=0.0.20 langgraph>=0.0.20 chromadb>=0.4.22 sqlalchemy>=2.0.0 python-dotenv>=1.0.0

print("✓ Installation complete!")

## Step 2: Set OpenAI API Key

**IMPORTANT:** Replace with your actual OpenAI API key!

In [ ]:
import os

# ⚠️ REPLACE WITH YOUR ACTUAL API KEY
os.environ['OPENAI_API_KEY'] = 'sk-proj-YOUR-KEY-HERE'

# Verify key is set
if os.environ.get('OPENAI_API_KEY', '').startswith('sk-'):
    print("✓ API key configured")
else:
    print("⚠️ WARNING: Please set your actual OpenAI API key above!")

## Step 3: Test Python Syntax (No API Needed)

In [ ]:
import sys
import ast

print("Testing Python files for syntax errors...\n")
print("=" * 60)

test_files = ['agent.py', 'tools.py', 'models/energy.py']
all_passed = True

for file_path in test_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()
        ast.parse(code)
        print(f"✓ {file_path}: Valid Python syntax")
    except SyntaxError as e:
        print(f"✗ {file_path}: Syntax error at line {e.lineno}")
        all_passed = False
    except Exception as e:
        print(f"? {file_path}: {str(e)}")
        all_passed = False

print("=" * 60)
if all_passed:
    print("\n✓ All syntax tests passed!")
else:
    print("\n⚠️ Some tests failed")

## Step 4: Initialize Database with Sample Data

Creates SQLite database with 90 days of energy usage and solar generation data.

In [ ]:
from models.energy import init_db, get_session, EnergyUsage, SolarGeneration
from datetime import datetime, timedelta
import random

print("Initializing database...")
init_db()

session = get_session()

# Generate 90 days of data
devices = ['hvac', 'water_heater', 'ev_charger', 'lighting', 'appliances']
start_date = datetime.now() - timedelta(days=90)

print("Generating sample data...")
for day in range(90):
    current_date = start_date + timedelta(days=day)
    
    # Energy usage (24 hourly records per device)
    for hour in range(24):
        timestamp = current_date + timedelta(hours=hour)
        for device in devices:
            # Realistic consumption patterns
            if device == 'hvac':
                consumption = random.uniform(2.5, 4.5)
            elif device == 'water_heater':
                consumption = random.uniform(1.0, 2.5)
            elif device == 'ev_charger':
                consumption = random.uniform(0, 7.0) if 22 <= hour or hour < 6 else 0
            elif device == 'lighting':
                consumption = random.uniform(0.3, 1.2) if 18 <= hour or hour < 7 else random.uniform(0.1, 0.4)
            else:  # appliances
                consumption = random.uniform(0.5, 2.0)
            
            usage = EnergyUsage(timestamp=timestamp, device=device, consumption_kwh=consumption)
            session.add(usage)
    
    # Solar generation (12 hours: 6 AM to 6 PM)
    for hour in range(6, 18):
        timestamp = current_date + timedelta(hours=hour)
        # Peak generation around noon
        hour_factor = 1 - abs(12 - hour) / 6
        generation = random.uniform(3.0, 6.0) * hour_factor
        efficiency = random.uniform(0.85, 0.95)
        
        solar = SolarGeneration(timestamp=timestamp, generation_kwh=generation, panel_efficiency=efficiency)
        session.add(solar)

session.commit()
session.close()

print(f"\n✓ Database initialized with 90 days of data")
print(f"  - Energy usage records: {90 * 24 * len(devices):,}")
print(f"  - Solar generation records: {90 * 12:,}")

## Step 5: Create Vector Store from Knowledge Documents

Processes 7 knowledge documents into ChromaDB for RAG retrieval.

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

print("Loading knowledge documents...")

documents_dir = "data/documents"
document_files = [
    "tip_device_best_practices.txt",
    "tip_energy_savings.txt",
    "hvac_optimization.txt",
    "smart_home_automation.txt",
    "renewable_energy_integration.txt",
    "seasonal_energy_management.txt",
    "energy_storage_optimization.txt"
]

all_texts = []
for filename in document_files:
    filepath = os.path.join(documents_dir, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
        all_texts.append(content)
        print(f"  ✓ Loaded {filename} ({len(content)} chars)")

# Split into chunks
print("\nSplitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = []
for text in all_texts:
    chunks.extend(text_splitter.split_text(text))

print(f"  ✓ Created {len(chunks)} chunks")

# Create embeddings and vector store
print("\nCreating vector store (this may take 1-2 minutes)...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("\n✓ Vector store created successfully!")
print(f"  - Total chunks indexed: {len(chunks)}")
print(f"  - Storage location: ./chroma_db")

## Step 6: Quick Agent Test

Tests the agent with a simple query to verify everything works.

In [ ]:
from agent import create_agent
from datetime import datetime

print("Creating agent...")
agent = create_agent()

print("\n" + "=" * 60)
print("Testing agent with sample query...")
print("=" * 60 + "\n")

query = "What are my top 3 energy-consuming devices today?"
print(f"User: {query}\n")

response = agent.chat(query)
print(f"Agent: {response}")

print("\n" + "=" * 60)
print("✓ Agent test complete!")
print("=" * 60)

## Step 7: Run Comprehensive Evaluation

Tests 8 different scenarios to validate all agent capabilities.

In [ ]:
# Test cases covering all capabilities
test_cases = [
    {
        "name": "Weather + Solar Analysis",
        "query": "What's the weather forecast for San Francisco this week, and how will it affect my solar generation?",
        "expected_tools": ["get_weather_forecast", "query_solar_generation"]
    },
    {
        "name": "EV Charging Optimization",
        "query": "When should I charge my electric vehicle to save money?",
        "expected_tools": ["get_electricity_prices", "search_energy_tips"]
    },
    {
        "name": "HVAC Efficiency",
        "query": "How can I make my HVAC system more efficient?",
        "expected_tools": ["search_energy_tips"]
    },
    {
        "name": "Usage Analysis",
        "query": "Show me my energy consumption for the past week",
        "expected_tools": ["query_energy_usage"]
    },
    {
        "name": "Seasonal Tips",
        "query": "What energy-saving strategies should I use this winter?",
        "expected_tools": ["search_energy_tips"]
    },
    {
        "name": "Battery Storage",
        "query": "Should I install battery storage with my solar panels?",
        "expected_tools": ["search_energy_tips"]
    },
    {
        "name": "Multi-Tool Query",
        "query": "Based on my usage last month and current electricity prices, how much can I save by shifting consumption to off-peak hours?",
        "expected_tools": ["query_energy_usage", "get_electricity_prices"]
    },
    {
        "name": "Price Analysis",
        "query": "What are the electricity prices for this week?",
        "expected_tools": ["get_electricity_prices"]
    }
]

print("\n" + "=" * 80)
print("COMPREHENSIVE EVALUATION - 8 Test Cases")
print("=" * 80 + "\n")

results = []
for i, test in enumerate(test_cases, 1):
    print(f"\n{'─' * 80}")
    print(f"Test {i}/8: {test['name']}")
    print(f"{'─' * 80}")
    print(f"\nQuery: {test['query']}\n")
    
    try:
        response = agent.chat(test['query'])
        print(f"Response: {response}\n")
        
        # Check if response is reasonable
        success = len(response) > 50 and "error" not in response.lower()
        results.append({"test": test['name'], "success": success})
        
        if success:
            print("✓ Test passed")
        else:
            print("⚠️ Response seems too short or contains errors")
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        results.append({"test": test['name'], "success": False})

# Summary
print("\n" + "=" * 80)
print("EVALUATION SUMMARY")
print("=" * 80)

passed = sum(1 for r in results if r['success'])
total = len(results)

for result in results:
    status = "✓" if result['success'] else "✗"
    print(f"{status} {result['test']}")

print(f"\n{'─' * 80}")
print(f"Total: {passed}/{total} tests passed ({passed/total*100:.1f}%)")
print("=" * 80)

## ✓ Testing Complete!

If all tests passed, your EcoHome Energy Advisor is working correctly!

### Next Steps:

1. **Try custom queries**: Create new cells and test your own questions
2. **Explore notebooks**: Check out the other notebooks in the repo
3. **Local setup**: If needed, follow README.md for local installation

### Common Issues:

- **API Rate Limits**: If you hit OpenAI rate limits, wait a minute and retry
- **Database locked**: Restart runtime if SQLite shows lock errors
- **Module not found**: Rerun Step 1 to reinstall dependencies

# EcoHome Energy Advisor - Google Colab Quick Start

This notebook sets up and tests the EcoHome agent in Google Colab.

⚠️ **Ignore dependency conflict warnings** - they're from Colab's internal packages and won't affect functionality.

## Step 1: Clone Repository and Install Dependencies

In [ ]:
# Clone repository
!git clone https://github.com/DipakBagal/udacity-ecohome-solutions.git
%cd udacity-ecohome-solutions

# Install dependencies (ignore version conflict warnings)
!pip install -q langchain>=0.1.0 langchain-openai>=0.0.5 langchain-community>=0.0.20 langgraph>=0.0.20 chromadb>=0.4.22 sqlalchemy>=2.0.0 python-dotenv>=1.0.0

print("✓ Installation complete!")

## Step 2: Set OpenAI API Key

**IMPORTANT:** Replace with your actual OpenAI API key!

In [ ]:
import os

# ⚠️ REPLACE WITH YOUR ACTUAL API KEY
os.environ['OPENAI_API_KEY'] = 'sk-proj-YOUR-KEY-HERE'

# Verify key is set
if os.environ.get('OPENAI_API_KEY', '').startswith('sk-'):
    print("✓ API key configured")
else:
    print("⚠️ WARNING: Please set your actual OpenAI API key above!")

## Step 3: Test Python Syntax (No API Needed)

In [ ]:
import sys
import ast

print("Testing Python files for syntax errors...\n")
print("=" * 60)

test_files = ['agent.py', 'tools.py', 'models/energy.py']
all_passed = True

for file_path in test_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()
            ast.parse(code)
        print(f"✓ {file_path}: Syntax OK")
    except SyntaxError as e:
        print(f"✗ {file_path}: Syntax Error at line {e.lineno}: {e.msg}")
        all_passed = False
    except Exception as e:
        print(f"✗ {file_path}: Error - {e}")
        all_passed = False

print("\n" + "=" * 60)
if all_passed:
    print("✓ All syntax checks passed!")
else:
    print("✗ Some syntax errors found")

## Step 4: Test Imports

In [ ]:
print("Testing imports...\n")
print("=" * 60)

imports_to_test = [
    ('LangChain Core', 'from langchain_core.messages import BaseMessage'),
    ('LangChain OpenAI', 'from langchain_openai import ChatOpenAI'),
    ('LangChain Community', 'from langchain_community.vectorstores import Chroma'),
    ('LangGraph', 'from langgraph.graph import StateGraph'),
    ('ChromaDB', 'import chromadb'),
    ('SQLAlchemy', 'from sqlalchemy import create_engine'),
    ('Python dotenv', 'from dotenv import load_dotenv'),
]

all_imports_ok = True
for name, import_stmt in imports_to_test:
    try:
        exec(import_stmt)
        print(f"✓ {name}: OK")
    except Exception as e:
        print(f"✗ {name}: Failed - {e}")
        all_imports_ok = False

print("\n" + "=" * 60)
if all_imports_ok:
    print("✓ All imports successful!")
else:
    print("✗ Some imports failed")

## Step 5: Test Database Models

In [ ]:
print("Testing database models...\n")
print("=" * 60)

try:
    from models.energy import EnergyUsage, SolarGeneration, init_db, get_session
    print("✓ Models imported successfully")
    
    # Test database initialization
    engine = init_db("test_ecohome.db")
    print("✓ Database initialized")
    
    # Test session creation
    session = get_session("test_ecohome.db")
    session.close()
    print("✓ Database session created")
    
    # Clean up test database
    import os
    if os.path.exists("test_ecohome.db"):
        os.remove("test_ecohome.db")
    
    print("\n" + "=" * 60)
    print("✓ Database models working correctly!")
    
except Exception as e:
    print(f"\n✗ Database test failed: {e}")
    import traceback
    traceback.print_exc()

## Step 6: Initialize Database with Sample Data

In [ ]:
print("Generating sample data (this may take a minute)...\n")
print("=" * 60)

from datetime import datetime, timedelta
import random
from models.energy import EnergyUsage, SolarGeneration, init_db, get_session

# Initialize database
engine = init_db("ecohome.db")
session = get_session("ecohome.db")

try:
    # Generate 30 days of data (faster for Colab)
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=30)
    
    current_date = start_date
    usage_count = 0
    solar_count = 0
    
    while current_date <= end_date:
        # Energy usage
        usage = EnergyUsage(
            timestamp=current_date,
            total_kwh=round(random.uniform(30, 50), 2),
            hvac_kwh=round(random.uniform(10, 20), 2),
            appliances_kwh=round(random.uniform(5, 12), 2),
            ev_charging_kwh=round(random.uniform(5, 15), 2),
            other_kwh=round(random.uniform(3, 8), 2)
        )
        session.add(usage)
        usage_count += 1
        
        # Solar generation
        generated = round(random.uniform(15, 30), 2)
        solar = SolarGeneration(
            timestamp=current_date,
            generated_kwh=generated,
            self_consumed_kwh=round(generated * 0.5, 2),
            exported_kwh=round(generated * 0.3, 2),
            battery_stored_kwh=round(generated * 0.2, 2)
        )
        session.add(solar)
        solar_count += 1
        
        current_date += timedelta(days=1)
    
    session.commit()
    print(f"✓ Generated {usage_count} energy usage records")
    print(f"✓ Generated {solar_count} solar generation records")
    
except Exception as e:
    session.rollback()
    print(f"✗ Error generating data: {e}")
finally:
    session.close()

print("\n" + "=" * 60)
print("✓ Database ready!")

## Step 7: Test Tools (Without RAG)

In [ ]:
print("Testing tools...\n")
print("=" * 60)

from tools import get_weather_forecast, get_electricity_prices, query_energy_usage, query_solar_generation
from datetime import datetime, timedelta

# Test weather forecast
try:
    result = get_weather_forecast.invoke({"location": "San Francisco", "days": 3})
    print("✓ Weather forecast tool working")
    print(f"  Sample: {result[:100]}...\n")
except Exception as e:
    print(f"✗ Weather forecast failed: {e}\n")

# Test electricity prices
try:
    result = get_electricity_prices.invoke({})
    print("✓ Electricity pricing tool working")
    print(f"  Sample: {result[:100]}...\n")
except Exception as e:
    print(f"✗ Electricity pricing failed: {e}\n")

# Test energy usage query
try:
    end = datetime.now().strftime("%Y-%m-%d")
    start = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
    result = query_energy_usage.invoke({"start_date": start, "end_date": end, "aggregate_by": "day"})
    print("✓ Energy usage query tool working")
    print(f"  Sample: {result[:100]}...\n")
except Exception as e:
    print(f"✗ Energy usage query failed: {e}\n")

# Test solar generation query
try:
    result = query_solar_generation.invoke({"start_date": start, "end_date": end})
    print("✓ Solar generation query tool working")
    print(f"  Sample: {result[:100]}...\n")
except Exception as e:
    print(f"✗ Solar generation query failed: {e}\n")

print("=" * 60)
print("✓ All tools tested!")

## Step 8: Initialize RAG Vector Store

In [ ]:
print("Initializing RAG vector store (this takes 1-2 minutes)...\n")
print("=" * 60)

try:
    from tools import initialize_vector_store
    
    vector_store = initialize_vector_store("./chroma_db")
    print("✓ Vector store initialized")
    
    # Test search
    results = vector_store.similarity_search("HVAC optimization", k=1)
    print(f"✓ Vector store search working")
    print(f"  Found {len(results)} result(s)")
    
    print("\n" + "=" * 60)
    print("✓ RAG system ready!")
    
except Exception as e:
    print(f"\n✗ RAG initialization failed: {e}")
    print("\nThis is expected if OpenAI API key is not set or invalid.")
    import traceback
    traceback.print_exc()

## Step 9: Test Agent (Requires Valid API Key)

In [ ]:
print("Testing agent...\n")
print("=" * 60)

try:
    from agent import create_agent
    
    # Create agent
    agent = create_agent(model_name="gpt-4o-mini", temperature=0.7)
    print("✓ Agent created successfully")
    
    # Test simple query
    print("\nTesting with sample query...\n")
    response = agent.chat(
        "What's the weather forecast for the next 3 days in San Francisco?",
        thread_id="colab_test"
    )
    
    print("Agent Response:")
    print("-" * 60)
    print(response)
    print("-" * 60)
    
    print("\n" + "=" * 60)
    print("✓ Agent is working correctly!")
    
except Exception as e:
    print(f"\n✗ Agent test failed: {e}")
    print("\nCommon causes:")
    print("  1. Invalid or missing OpenAI API key")
    print("  2. API quota exceeded")
    print("  3. Network connectivity issues")
    import traceback
    traceback.print_exc()

## Step 10: Interactive Testing

In [ ]:
# Interactive chat with the agent
from agent import create_agent

agent = create_agent(model_name="gpt-4o-mini", temperature=0.7)

print("EcoHome Energy Advisor - Interactive Mode")
print("=" * 60)
print("Try these example queries:")
print("  - What's the current electricity pricing?")
print("  - Analyze my energy usage for the past week")
print("  - How is my solar system performing?")
print("  - Give me tips to reduce HVAC costs")
print("\nEnter your query below:")
print("=" * 60)

# Get user input
user_query = input("You: ")

if user_query.strip():
    print("\nEcoHome Agent:")
    print("-" * 60)
    try:
        response = agent.chat(user_query, thread_id="interactive")
        print(response)
    except Exception as e:
        print(f"Error: {e}")
    print("-" * 60)

## Summary

✅ **What was tested:**
1. Python syntax validation
2. Package imports
3. Database models and operations
4. Tool functionality (weather, pricing, queries)
5. RAG vector store setup
6. Agent creation and execution

📊 **System Status:**
- Database: 30 days of sample data
- Vector Store: 7 knowledge base documents
- Tools: 5 specialized tools ready
- Agent: LangGraph workflow with conversation memory

🎯 **Next Steps:**
- Test with more complex queries
- Explore different agent scenarios
- Customize knowledge base
- Add real API integrations

---

**Repository:** https://github.com/DipakBagal/udacity-ecohome-solutions

## Step 1: Clone Repository and Install Dependencies

In [ ]:
# Clone repository
!git clone https://github.com/DipakBagal/udacity-ecohome-solutions.git
%cd udacity-ecohome-solutions

# Install dependencies (ignore version conflict warnings)
!pip install -q langchain>=0.1.0 langchain-openai>=0.0.5 langchain-community>=0.0.20 langgraph>=0.0.20 chromadb>=0.4.22 sqlalchemy>=2.0.0 python-dotenv>=1.0.0

print("✓ Installation complete!")

## Step 2: Set OpenAI API Key

**IMPORTANT:** Replace with your actual OpenAI API key!

In [ ]:
import os

# ⚠️ REPLACE WITH YOUR ACTUAL API KEY
os.environ['OPENAI_API_KEY'] = 'sk-proj-YOUR-KEY-HERE'

# Verify key is set
if os.environ.get('OPENAI_API_KEY', '').startswith('sk-'):
    print("✓ API key configured")
else:
    print("⚠️ WARNING: Please set your actual OpenAI API key above!")

## Step 3: Quick Agent Test

Tests the agent with a simple query to verify everything works.

In [ ]:
# Run the existing setup and evaluation notebook
!jupyter nbconvert --to notebook --execute 01_db_setup.ipynb --output /tmp/01_executed.ipynb
!jupyter nbconvert --to notebook --execute 02_rag_setup.ipynb --output /tmp/02_executed.ipynb
!jupyter nbconvert --to notebook --execute 03_run_and_evaluate.ipynb --output /tmp/03_executed.ipynb

print("\n✓ All setup and tests complete! Check the output above for results.")